In [ ]:
import pandas as pd 
from chat_data import get_chat_data
pd_data= get_chat_data("../../backend/data/webui.db")

2025-07-16 08:06:23,887 - INFO - 数据库引擎创建成功。
2025-07-16 08:06:23,964 - INFO - 查询成功执行，返回 1125 行。
2025-07-16 08:06:23,965 - INFO - 成功读取chat表数据，共 1125 条记录


In [ ]:
pd_data.groupby(['user_name']).count()

,chat_id,chat_user_id,chat_title,chat_model,last_chat_model,chat_created_at,chat_updated_at,role,model,models,message_id,parentId,last_child_id,childrenIds,created_at,content,respond_content
user_name,,,,,,,,,,,,,,,,,
xuchengba,41,41,41,41,41,41,41,41,0,41,41,24,41,41,41,41,41
zongyuting-,109,109,109,109,109,109,109,109,0,109,109,98,109,109,109,109,109
义政妈,11,11,11,11,11,11,11,11,0,11,11,0,11,11,11,11,11
刘桦映,2,2,2,2,2,2,2,2,0,2,2,0,2,2,2,2,2
华娟冰,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1
呼呼huhu～,96,96,96,96,96,96,96,96,0,96,96,3,96,96,96,96,96
地瓜妈,388,388,388,388,388,388,388,388,0,388,388,111,388,388,388,388,388
宋园梦,85,85,85,85,85,85,85,85,0,85,85,29,85,85,85,85,85
康复师X壹壹妈,543,543,543,543,543,543,543,543,0,543,543,76,543,543,543,543,543


In [ ]:
from chat_data import get_db_engine,_fetch_raw_chat_data,get_chat_data,_parse_chat_messages,_create_chat_view
engine = get_db_engine("/root/autodl-tmp/dev/open_webui/backend/data/webui.db")

chat_df = _fetch_raw_chat_data(engine)

chat_df=_parse_chat_messages(chat_df)

chat_view_df = _create_chat_view(chat_df)
queries = chat_df[chat_df.role == "assistant"].copy()

len(chat_view_df)


In [ ]:
import json 

import datetime
def timestamp_to_datetime(timestamp):
    if timestamp is None:
        return None
    return datetime.datetime.fromtimestamp(timestamp)
chat_data=[]
for _, r in chat_df.iterrows():
        chat_ = json.loads(r['chat'])
        user_id = r['user_id']
        user_name = r.get('name')
        
        if user_name in ['dali', 'cz', ' cz']:
            continue

        chat_model = chat_.get("models", [])
        chat_title = chat_.get("title")
        chat_id = chat_.get("id")

        for msg_id, hist in chat_.get("history", {}).get("messages", {}).items():
              chat_data.append({
                    "chat_id": chat_id,
                    "chat_user_id": user_id,
                    "chat_title": chat_title,
                    "chat_model": chat_model,
                    "last_chat_model": chat_model[-1] if chat_model else None,
                    "chat_created_at": timestamp_to_datetime(r['created_at']),
                    "chat_updated_at": timestamp_to_datetime(r['updated_at']),
                    "user_name": user_name,
                    "role": hist.get("role"),
                    "model": hist.get("model"),
                    "models": hist.get("models"),
                    "message_id": msg_id,
                    "parentId": hist.get("parentId"),
                    "last_child_id": hist.get("childrenIds", [])[-1] if hist.get("childrenIds") else None,
                    "childrenIds": hist.get("childrenIds"),
                    "created_at": timestamp_to_datetime(hist.get("timestamp")),
                    "content": hist.get("content")
                })
              
len(chat_data)

chat_data_pd = pd.DataFrame(chat_data)
chat_data_pd.info()
            